In [ ]:
#!/usr/bin/env python
# _*_ coding: utf-8 _*_

import numpy as np


"""
    搜素局部最大值，抑制极大值
    前提：目标边界框列表及其对应的置信度得分列表，设定阈值，阈值用来删除重叠较大的边界框。
    IoU：intersection-over-union，即两个边界框的交集部分除以它们的并集。

      非极大值抑制的流程如下：

        根据置信度得分进行排序

        选择置信度最高的边界框添加到最终输出列表中，将其从边界框列表中删除

        计算所有边界框的面积

        计算置信度最高的边界框与其它候选框的IoU。

        删除IoU大于阈值的边界框

        重复上述过程，直至边界框列表为空。
    Non-max Suppression Algorithm
    
    @param list  Object candidate bounding boxes
    @param list  Confidence score of bounding boxes
    @param float IoU threshold

    @return Rest boxes after nms operation
"""
def nms(bounding_boxes, confidence_score, threshold):
    # If no bounding boxes, return empty list
    if len(bounding_boxes) == 0:
        return [], []

    # Bounding boxes
    boxes = np.array(bounding_boxes)

    # coordinates of bounding boxes
    start_x = boxes[:, 0]
    start_y = boxes[:, 1]
    end_x = boxes[:, 2]
    end_y = boxes[:, 3]
    # Confidence scores of bounding boxes
    score = np.array(confidence_score)

    # Picked bounding boxes
    picked_boxes = []
    picked_score = []

    # Compute areas of bounding boxes
    areas = (end_x - start_x + 1) * (end_y - start_y + 1)

    # Sort by confidence score of bounding boxes
    order = np.argsort(score)

    # Iterate bounding boxes
    while order.size > 0:
        # The index of largest confidence score
        index = order[-1]
        # Pick the bounding box with largest confidence score
        picked_boxes.append(bounding_boxes[index])
        picked_score.append(confidence_score[index])

        # Compute ordinates of intersection-over-union(IOU)
        # 计算交并比，x1,y1取最大值，x2,y2取最小值
        x1 = np.maximum(start_x[index], start_x[order[:-1]])
        x2 = np.minimum(end_x[index], end_x[order[:-1]])
        y1 = np.maximum(start_y[index], start_y[order[:-1]])
        y2 = np.minimum(end_y[index], end_y[order[:-1]])

        # Compute areas of intersection-over-union
        w = np.maximum(0.0, x2 - x1 + 1)
        h = np.maximum(0.0, y2 - y1 + 1)
        intersection = w * h

        # Compute the ratio between intersection and union
        iou = intersection / (areas[index] + areas[order[:-1]] - intersection)

        left = np.where(iou < threshold)
        order = order[left]

    return picked_boxes, picked_score



# Bounding boxes
bounding_boxes = [(187, 82, 337, 317), (150, 67, 305, 282), (246, 121, 368, 304)]
confidence_score = [0.9, 0.75, 0.8]

# IoU threshold
threshold = 0.4

# Run non-max suppression algorithm
picked_boxes, picked_score = nms(bounding_boxes, confidence_score, threshold)
print("picked_boxes:",picked_boxes)
print("picked_score:",picked_score)